In [1]:
#Ici j'ai:
# 1)continué mon analyse descriptive
# 2)fait feature engineering pour les categories avec le pire score (funny,conpiracy,AskReddit)
# 3) feature engineering general pour toutest les categories avec chiffres, http ou .gif .jpg dans le commentaries
# 4) testes avec LienarSVC
# 5) dummy search pour les hyperparametres avec scikit-optimizer
# 6) Baysean search pour les hyperparametres avec scikit-optimizer 

In [1]:

from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

from wordcloud import WordCloud, STOPWORDS

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV

import warnings
warnings.simplefilter(action='ignore')

%matplotlib inline

In [2]:


wc_df = pd.read_csv('train.csv') 
comment_data =wc_df

comment_data['prep'] = comment_data['comment'].str.replace(r'[^\w\s]+', '') # tira td que nao palavra e espaco em branco
comment_data['prep'] = comment_data['prep'].str.lower()
comment_data['prep'] = comment_data['prep'].str.replace('(\d+)', ' num ')
#comment_data['prep'] = comment_data['prep'].str.replace(r'http(?<=http).*', ' ')
comment_data['prep'] = comment_data['prep'].str.replace(r'\s+', " ")

In [3]:
# natural language toolkit
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tag import pos_tag_sents
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
tt = TweetTokenizer()
def lemmatize_col(row):
    row = tt.tokenize(row)
    return ' '.join([lemmatizer.lemmatize(w) for w in row])

comment_data['prep'] = comment_data['prep'].apply(lemmatize_col)
#test_data['prep'] = test_data['prep'].apply(lemmatize_col) 

# stopwords
stop = stopwords.words('english')
comment_data['prep'] = comment_data['prep'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#test_data['prep'] = test_data['prep'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [4]:
wc_df.head()

,id,comment,subreddit,prep
0,0,I think prestige points should not expire ever...,leagueoflegends,think prestige point expire ever skin buy avai...
1,1,Whats going to happen with them if they will b...,europe,whats going happen refused asilum appeal
2,2,Anecdotal evidence is anecdotal. Clearly by “e...,gameofthrones,anecdotal evidence anecdotal clearly everyone ...
3,3,"Look dude, with all due respect, your music is...",Music,look dude due respect music isnt people look l...
4,4,Hope he gets the doomhammer back!,wow,hope get doomhammer back


In [5]:
imur=wc_df.loc[wc_df['comment'].str.contains('imur')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
imur

,id,comment,prep
subreddit,,,
nba,3,3,3


In [6]:


picture=wc_df.loc[wc_df['comment'].str.contains('picture')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
picture


,id,comment,prep
subreddit,,,
gameofthrones,25,25,25
conspiracy,21,21,21
europe,21,21,21
trees,19,19,19
movies,19,19,19
funny,18,18,18
AskReddit,12,12,12
canada,12,12,12
worldnews,11,11,11


In [7]:


meme=wc_df.loc[wc_df['comment'].str.contains('meme')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
meme


,id,comment,prep
subreddit,,,
anime,22,22,22
gameofthrones,19,19,19
wow,14,14,14
conspiracy,13,13,13
Overwatch,12,12,12
GlobalOffensive,12,12,12
leagueoflegends,12,12,12
europe,11,11,11
baseball,10,10,10


In [8]:

lol=wc_df.loc[wc_df['comment'].str.contains('lol')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
lol


,id,comment,prep
subreddit,,,
nba,106,106,106
trees,90,90,90
GlobalOffensive,83,83,83
leagueoflegends,71,71,71
nfl,67,67,67
hockey,60,60,60
Overwatch,53,53,53
soccer,53,53,53
anime,52,52,52


In [9]:

rs=wc_df.loc[wc_df['comment'].str.contains('haha')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
rs


,id,comment,prep
subreddit,,,
trees,38,38,38
gameofthrones,16,16,16
funny,14,14,14
leagueoflegends,14,14,14
hockey,13,13,13
wow,13,13,13
Music,13,13,13
Overwatch,12,12,12
soccer,12,12,12


In [10]:
why=wc_df.loc[wc_df['comment'].str.contains('why')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
why

,id,comment,prep
subreddit,,,
conspiracy,144,144,144
worldnews,126,126,126
anime,116,116,116
europe,116,116,116
Overwatch,112,112,112
canada,112,112,112
gameofthrones,107,107,107
movies,106,106,106
leagueoflegends,105,105,105


In [11]:
https=wc_df.loc[wc_df['comment'].str.contains('https')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
https

,id,comment,prep
subreddit,,,
anime,389,389,389
Music,205,205,205
conspiracy,203,203,203
europe,172,172,172
worldnews,144,144,144
canada,118,118,118
wow,100,100,100
GlobalOffensive,91,91,91
funny,90,90,90


In [12]:
http=wc_df.loc[wc_df['comment'].str.contains('http')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)

In [13]:
http

,id,comment,prep
subreddit,,,
anime,392,392,392
conspiracy,216,216,216
Music,205,205,205
europe,183,183,183
worldnews,152,152,152
canada,130,130,130
wow,103,103,103
funny,99,99,99
hockey,94,94,94


In [14]:
gif=wc_df.loc[wc_df['comment'].str.contains('gif')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)

In [15]:
gif

,id,comment,prep
subreddit,,,
anime,19,19,19
funny,14,14,14
hockey,10,10,10
nfl,10,10,10
leagueoflegends,10,10,10
soccer,9,9,9
wow,7,7,7
europe,6,6,6
Music,5,5,5


In [16]:
jpg=wc_df.loc[wc_df['comment'].str.contains('jpg')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
jpg

,id,comment,prep
subreddit,,,
anime,55,55,55
baseball,13,13,13
conspiracy,13,13,13
europe,11,11,11
gameofthrones,9,9,9
wow,9,9,9
Overwatch,8,8,8
funny,8,8,8
hockey,7,7,7


In [17]:
lmao=wc_df.loc[wc_df['comment'].str.contains('lmao')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
lmao

,id,comment,prep
subreddit,,,
nba,50,50,50
soccer,32,32,32
leagueoflegends,29,29,29
GlobalOffensive,27,27,27
trees,26,26,26
nfl,24,24,24
baseball,21,21,21
hockey,20,20,20
anime,17,17,17


In [18]:
omg=wc_df.loc[wc_df['comment'].str.contains('omg')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
omg

,id,comment,prep
subreddit,,,
wow,6,6,6
GlobalOffensive,3,3,3
Overwatch,3,3,3
baseball,2,2,2
hockey,2,2,2
nba,2,2,2
trees,2,2,2
AskReddit,1,1,1
canada,1,1,1


In [19]:
whats=wc_df.loc[wc_df['comment'].str.contains('whats')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
whats

,id,comment,prep
subreddit,,,
europe,9,9,9
canada,6,6,6
worldnews,5,5,5
trees,5,5,5
GlobalOffensive,5,5,5
gameofthrones,4,4,4
anime,4,4,4
conspiracy,4,4,4
soccer,4,4,4


In [20]:
what=wc_df.loc[wc_df['comment'].str.contains('what')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
what

,id,comment,prep
subreddit,,,
canada,360,360,360
worldnews,351,351,351
conspiracy,346,346,346
gameofthrones,293,293,293
anime,291,291,291
europe,289,289,289
movies,284,284,284
wow,256,256,256
Overwatch,248,248,248


In [21]:
since=wc_df.loc[wc_df['comment'].str.contains('since')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
since

,id,comment,prep
subreddit,,,
wow,129,129,129
anime,122,122,122
GlobalOffensive,82,82,82
leagueoflegends,78,78,78
Overwatch,76,76,76
europe,69,69,69
Music,68,68,68
canada,68,68,68
worldnews,67,67,67


In [22]:
how=wc_df.loc[wc_df['comment'].str.contains('how')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
how

,id,comment,prep
subreddit,,,
anime,518,518,518
gameofthrones,473,473,473
movies,323,323,323
conspiracy,321,321,321
Music,291,291,291
canada,273,273,273
worldnews,267,267,267
europe,266,266,266
wow,265,265,265


In [23]:
which=wc_df.loc[wc_df['comment'].str.contains('which')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
which

,id,comment,prep
subreddit,,,
europe,223,223,223
canada,148,148,148
wow,130,130,130
Music,120,120,120
anime,116,116,116
worldnews,116,116,116
conspiracy,116,116,116
movies,112,112,112
leagueoflegends,109,109,109


In [24]:
who=wc_df.loc[wc_df['comment'].str.contains('who')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
who

,id,comment,prep
subreddit,,,
conspiracy,317,317,317
canada,316,316,316
worldnews,287,287,287
movies,260,260,260
gameofthrones,246,246,246
anime,235,235,235
Music,234,234,234
europe,218,218,218
Overwatch,215,215,215


In [25]:
ask=wc_df.loc[wc_df['comment'].str.contains('ask')].groupby(['subreddit']).count().sort_values(by='prep', ascending=False)
ask

,id,comment,prep
subreddit,,,
worldnews,80,80,80
AskReddit,74,74,74
canada,72,72,72
anime,69,69,69
nba,62,62,62
conspiracy,58,58,58
europe,53,53,53
trees,52,52,52
gameofthrones,49,49,49


In [26]:
wc_df_wow=wc_df.loc[wc_df['subreddit']=='wow']
wc_df_wow

,id,comment,subreddit,prep
4,4,Hope he gets the doomhammer back!,wow,hope get doomhammer back
57,57,Disc is also rewarded by your group knowing en...,wow,disc also rewarded group knowing encounter wel...
60,60,I mean I had every class at 100 and 110 and di...,wow,mean every class num num leveling process ha c...
70,70,The Ebon Blade vowed to fight Arthas for reven...,wow,ebon blade vowed fight arthas revenge cause se...
74,74,"I don't have any money at all due to 0 income,...",wow,dont money due num income
...,...,...,...,...
59955,59955,&gt; Do you want infinite dragonflights? \n\nY...,wow,gt want infinite dragonflights yes please
59957,59957,no. they're just different! artifacts are that...,wow,theyre different artifact goldybrown colour ic...
59974,59974,Isn't 4.4 the stock boost clock of the 4790K?,wow,isnt num stock boost clock num k
59978,59978,"Stoned elves man, get it right",wow,stoned elf man get right


In [27]:
wc_df_wow_words=wc_df_wow['prep'].str.split(expand=True).stack().value_counts()
wc_df_wow_words

num          1705
wa            753
like          614
get           455
one           425
             ... 
badlazy         1
flesh           1
illogical       1
crotch          1
dmgor           1
Length: 9357, dtype: int64

In [28]:
wc_df_lol=wc_df.loc[wc_df['subreddit']=='leagueoflegends']
wc_df_lol

,id,comment,subreddit,prep
0,0,I think prestige points should not expire ever...,leagueoflegends,think prestige point expire ever skin buy avai...
33,33,Only reason anyone even remembers Marth and Ik...,leagueoflegends,reason anyone even remembers marth ike cause s...
86,86,"Only the affected team should get to chose, mi...",leagueoflegends,affected team get chose misfit team doesnt get...
93,93,i love the yasuo crying face,leagueoflegends,love yasuo cry face
150,150,Eve mains hate Blood Moon because it was a ski...,leagueoflegends,eve main hate blood moon wa skin always wanted...
...,...,...,...,...
59917,59917,And they are a totally different beast when it...,leagueoflegends,totally different beast come bo num
59926,59926,Rick Fox is actually at a TV show set right no...,leagueoflegends,rick fox actually tv show set right guess mean...
59960,59960,Renekton doesn't get outscaled midgame with Sh...,leagueoflegends,renekton doesnt get outscaled midgame shojin f...
59962,59962,Why do teams not clear the blast plant when do...,leagueoflegends,team clear blast plant baron


In [29]:
wc_df_lol_words=wc_df_lol['prep'].str.split(expand=True).stack().value_counts()
wc_df_lol_words

num            1823
game            677
wa              562
like            497
get             420
               ... 
explained         1
allstars          1
predictable       1
pistol            1
piglet            1
Length: 7722, dtype: int64

In [30]:
wc_df_funny=wc_df.loc[wc_df['subreddit']=='funny']
wc_df_funny

,id,comment,subreddit,prep
9,9,https://i.imgur.com/DUdy0KL.jpg,funny,httpsiimgurcomdudy num kljpg
23,23,All music can broken down to structures. Point...,funny,music broken structure pointing structure kind...
24,24,I too bring my bow into the hospital after I’v...,funny,bring bow hospital ive shot dad shoulder arrow...
45,45,Go to the gym and eat a healthy diet. \n\nAnd ...,funny,go gym eat healthy diet get sunlight
49,49,"Politicians too. Willing to roast each other, ...",funny,politician willing roast oh shit dont sound an...
...,...,...,...,...
59883,59883,"Maybe hold your reaction till you get facts, h...",funny,maybe hold reaction till get fact
59906,59906,I would've taken my gun out and shot that ball...,funny,wouldve taken gun shot balloon wa making escap...
59915,59915,I most certainly would. 4 backhoes sitting in ...,funny,certainly would num backhoe sitting yard komat...
59983,59983,The only time colon broke that I can easily re...,funny,time colon broke easily recall watching animal...


In [31]:
wc_df_conspiracy=wc_df.loc[wc_df['subreddit']=='conspiracy']
wc_df_conspiracy

,id,comment,subreddit,prep
11,11,Is it really that far fetched? It actually mak...,conspiracy,really far fetched actually make perfect sense...
14,14,Do you believe in ancestral spirit? Maybe this...,conspiracy,believe ancestral spirit maybe synchronicity h...
15,15,The JFK assassination.,conspiracy,jfk assassination
47,47,"Yes it was. Which I always found confusing, co...",conspiracy,yes wa always found confusing considering wore...
78,78,As the story goes they were waiting for Muelle...,conspiracy,story go waiting mueller wrap thats also fishy...
...,...,...,...,...
59931,59931,Those Twitter comments smh,conspiracy,twitter comment smh
59937,59937,I practice meditation. It has really helped me...,conspiracy,practice meditation ha really helped control a...
59942,59942,"[For the First Time Ever, Temperatures Reached...",conspiracy,first time ever temperature reached num degree...
59970,59970,Well... scooter guy. 80s. Vancouver BC. Down t...,conspiracy,well scooter guy num vancouver bc num u


In [32]:
wc_df_AskReddit=wc_df.loc[wc_df['subreddit']=='AskReddit']
wc_df_AskReddit

,id,comment,subreddit,prep
20,20,https://youtu.be/zQJcfkDjhRg,AskReddit,httpsyoutubezqjcfkdjhrg
41,41,That assumption one is doing a lot of work. Ea...,AskReddit,assumption one lot work earth need unique form...
65,65,I would fuck their mother,AskReddit,would fuck mother
67,67,Every dog is the best,AskReddit,every dog best
77,77,Big Rigs over the road racing \n\n\nET on ata...,AskReddit,big rig road racing et atari ride hell retribu...
...,...,...,...,...
59891,59891,Breaking bad. Definitely.,AskReddit,breaking bad definitely
59899,59899,There's a lot of articles out there. Gotta sea...,AskReddit,lot article gotta search engine one although a...
59990,59990,Children are still dying from preventable dise...,AskReddit,child still dying preventable disease around w...
59993,59993,What's a good costume for a muscular girl with...,AskReddit,whats good costume muscular girl visible ab


In [33]:
wc_df_funny_words=wc_df_funny['prep'].str.split(expand=True).stack().value_counts()
wc_df_funny_words[0:20]

num       770
wa        357
like      346
one       213
get       204
people    203
dont      198
im        181
would     166
thats     162
time      153
know      148
think     133
ha        124
thing     122
make      115
good      113
go        110
really    109
look      105
dtype: int64

In [34]:
wc_df_conspiracy_words=wc_df_conspiracy['prep'].str.split(expand=True).stack().value_counts()
wc_df_conspiracy_words[0:20]

num       1769
wa         750
people     564
like       502
dont       375
one        344
ha         332
would      330
think      308
get        301
trump      293
know       283
even       234
im         233
make       230
year       218
thing      217
thats      209
u          206
gt         199
dtype: int64

In [35]:
wc_df_AskReddit_words=wc_df_AskReddit['prep'].str.split(expand=True).stack().value_counts()
wc_df_AskReddit_words[0:20]

num       678
wa        674
like      388
get       311
one       301
people    300
would     263
im        256
dont      256
time      228
know      212
think     187
thing     185
make      183
go        160
ha        145
really    141
got       140
thats     140
year      139
dtype: int64

In [ ]:
wc_df_ALL_words=wc_df['prep'].str.split(expand=True).stack().value_counts()
wc_df_ALL_words[0:20]

In [36]:
for df_custom in [wc_df_funny,wc_df_conspiracy,wc_df_AskReddit,wc_df_lol,wc_df_wow]:

    text = ' '
    stopwords = set(STOPWORDS) 

    # iterate through the csv file 
    for x in df_custom.prep: 

        # typecaste each val to string 
        x = str(x) 

        # split the value 
        values = x.split() 

        # Converts each token into lowercase 
        for i in range(len(values)): 
            values[i] = values[i].lower() 

        for words in values: 
            text = text + words + ' '


    wc = WordCloud(max_words= 100,
                          width = 744, 
                          height = 544,
                          background_color ='white',
                          stopwords=stopwords, 
                          contour_width=3, 
                          contour_color='steelblue',
                          min_font_size = 10).generate(text) 

    # plot the WordCloud image    
    #plt.figure(figsize = (14, 14)) 
    #print(df_custom.subreddit)
    #plt.imshow(wc) 
    #plt.axis("off")
    #plt.savefig('../images/reddit_wordcloud.png')

In [37]:

  
text = ' '
stopwords = set(STOPWORDS) 
  
# iterate through the csv file 
for x in wc_df_funny[0:20].prep: 
      
    # typecaste each val to string 
    x = str(x) 
  
    # split the value 
    values = x.split() 
      
    # Converts each token into lowercase 
    for i in range(len(values)): 
        values[i] = values[i].lower() 
          
    for words in values: 
        text = text + words + ' '
  
  
wc = WordCloud(max_words= 100,
                      width = 744, 
                      height = 544,
                      background_color ='white',
                      stopwords=stopwords, 
                      contour_width=3, 
                      contour_color='steelblue',
                      min_font_size = 10).generate(text) 
  
# plot the WordCloud image                        
#plt.figure(figsize = (14, 14)) 
#plt.imshow(wc) 
#plt.axis("off")
#plt.savefig('../images/reddit_wordcloud.png')

In [3]:

import pandas as pd
import re
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2



import numpy as np
import pandas as pd
import scipy.sparse as sp
import matplotlib.pyplot as plt
%matplotlib inline



# natural language toolkit
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tag import pos_tag_sents
from nltk.stem import WordNetLemmatizer

# SciKit-Learn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from skopt import gp_minimize, forest_minimize, dummy_minimize

import pandas as pd

train = pd.read_csv('train.csv')


test = pd.read_csv('test.csv')


#train



In [4]:
#preprocessing


comment_data = train

#clean
comment_data['prep'] = comment_data['comment'].str.replace(r'[^\w\s]+', '') # tira td que nao palavra e espaco em branco
comment_data['prep'] = comment_data['prep'].str.lower()
comment_data['prep'] = comment_data['prep'].str.replace('(\d+)', ' num ')
#comment_data['prep'] = comment_data['prep'].str.replace(r'http(?<=http).*', ' ')
comment_data['prep'] = comment_data['prep'].str.replace(r'\s+', " ")
comment_data['prep'] = comment_data['prep'].str.replace(" +", " ")

#comment_data





test_data = pd.read_csv('test.csv')
#clean
test_data['prep'] = test_data['comment'].str.replace(r'[^\w\s]+', '')
test_data['prep'] = test_data['prep'].str.lower()
test_data['prep'] = test_data['prep'].str.replace('(\d+)', ' num ')
#test_data['prep'] = test_data['prep'].str.replace(r'http(?<=http).*', ' ')
test_data['prep'] = test_data['prep'].str.replace(r'\s+', " ")
test_data['prep'] = test_data['prep'].str.replace(" +", " ")

In [5]:
lemmatizer = WordNetLemmatizer()
tt = TweetTokenizer()
def lemmatize_col(row):
    row = tt.tokenize(row)
    return ' '.join([lemmatizer.lemmatize(w) for w in row])

comment_data['prep'] = comment_data['prep'].apply(lemmatize_col)
test_data['prep'] = test_data['prep'].apply(lemmatize_col) 

# stopwords
stop = stopwords.words('english')
comment_data['prep'] = comment_data['prep'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test_data['prep'] = test_data['prep'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

### make new features

In [6]:
#CRIANDO FEATURES NEW_COMMENT

train_funny=train.loc[train['subreddit']=='funny']
train_funny



,id,comment,subreddit,prep
9,9,https://i.imgur.com/DUdy0KL.jpg,funny,httpsiimgurcomdudy num kljpg
23,23,All music can broken down to structures. Point...,funny,music broken structure pointing structure kind...
24,24,I too bring my bow into the hospital after I’v...,funny,bring bow hospital ive shot dad shoulder arrow...
45,45,Go to the gym and eat a healthy diet. \n\nAnd ...,funny,go gym eat healthy diet get sunlight
49,49,"Politicians too. Willing to roast each other, ...",funny,politician willing roast oh shit dont sound an...
...,...,...,...,...
59883,59883,"Maybe hold your reaction till you get facts, h...",funny,maybe hold reaction till get fact
59906,59906,I would've taken my gun out and shot that ball...,funny,wouldve taken gun shot balloon wa making escap...
59915,59915,I most certainly would. 4 backhoes sitting in ...,funny,certainly would num backhoe sitting yard komat...
59983,59983,The only time colon broke that I can easily re...,funny,time colon broke easily recall watching animal...


In [7]:

train_Ask=train.loc[train['subreddit']=='AskReddit']
train_Ask

,id,comment,subreddit,prep
20,20,https://youtu.be/zQJcfkDjhRg,AskReddit,httpsyoutubezqjcfkdjhrg
41,41,That assumption one is doing a lot of work. Ea...,AskReddit,assumption one lot work earth need unique form...
65,65,I would fuck their mother,AskReddit,would fuck mother
67,67,Every dog is the best,AskReddit,every dog best
77,77,Big Rigs over the road racing \n\n\nET on ata...,AskReddit,big rig road racing et atari ride hell retribu...
...,...,...,...,...
59891,59891,Breaking bad. Definitely.,AskReddit,breaking bad definitely
59899,59899,There's a lot of articles out there. Gotta sea...,AskReddit,lot article gotta search engine one although a...
59990,59990,Children are still dying from preventable dise...,AskReddit,child still dying preventable disease around w...
59993,59993,What's a good costume for a muscular girl with...,AskReddit,whats good costume muscular girl visible ab


In [8]:
train_Ask['new_comment'] = np.where((train_Ask.prep.str.contains('how')|
                                     train_Ask.prep.str.contains('since') |
                                     train_Ask.prep.str.contains('http') |
                                     train_Ask.prep.str.contains('https') |
                                     train_Ask.prep.str.contains('ask') |
                                     train_Ask.prep.str.contains('who') |
                                     train_Ask.prep.str.contains('why') |
                                     train_Ask.prep.str.contains('which') |
                                     train_Ask.prep.str.contains('what') |
                                     
                                     train_Ask.prep.str.contains('jpg')|
                                       train_Ask.prep.str.contains('jpeg') |
                                       train_Ask.prep.str.contains('http') |

                                       train_Ask.prep.str.contains('https') |

                                       train_Ask.prep.str.contains('picture') |
                                       train_Ask.prep.str.contains('gif') |

                                     
                                     
                                     mexer pouco e so nas que estao ruins
                                     
                                     
                                      np.where((train.prep.str.contains('http') |
                                    # train.prep.str.contains('https') |    #nao precisa pois esta contido no http                               
                                            
                                       train.prep.str.contains('jpg')|
                                       train.prep.str.contains('jpeg') |
                                     

                                       train.prep.str.contains('picture') |
                                       train.prep.str.contains('gif') |
                                     
                                    ( train.prep.str.contains('http') & train.prep.str.contains('num')) |
                                     
                                  
                                      #((train.prep.str.contains('https') & (train.prep.str.contains('num')) |
                                     
                                        (train.prep.str.contains('jpg') & train.prep.str.contains('num')) |
                                          (train.prep.str.contains('jpeg') & train.prep.str.contains('num')) |
                                            (train.prep.str.contains('picture') & train.prep.str.contains('num')) |
                                              (train.prep.str.contains('gif') & train.prep.str.contains('num')) 
                                                
                                                
                                     


                                      ),'seems' + ' ' + train.subreddit, '.')

                                     
                                     
                                     


                                       train_Ask.prep.str.contains('num') 
                                   


                                      ),'is AskReddit', '.') 

train_Ask



/usr/lib/python3/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,comment,subreddit,prep,new_comment
20,20,https://youtu.be/zQJcfkDjhRg,AskReddit,httpsyoutubezqjcfkdjhrg,is AskReddit
41,41,That assumption one is doing a lot of work. Ea...,AskReddit,assumption one lot work earth need unique form...,is AskReddit
65,65,I would fuck their mother,AskReddit,would fuck mother,.
67,67,Every dog is the best,AskReddit,every dog best,.
77,77,Big Rigs over the road racing \n\n\nET on ata...,AskReddit,big rig road racing et atari ride hell retribu...,.
...,...,...,...,...,...
59891,59891,Breaking bad. Definitely.,AskReddit,breaking bad definitely,.
59899,59899,There's a lot of articles out there. Gotta sea...,AskReddit,lot article gotta search engine one although a...,is AskReddit
59990,59990,Children are still dying from preventable dise...,AskReddit,child still dying preventable disease around w...,.
59993,59993,What's a good costume for a muscular girl with...,AskReddit,whats good costume muscular girl visible ab,is AskReddit


In [9]:
train_funny['new_comment'] = np.where((train_funny.prep.str.contains('jpg')|
                                       train_funny.prep.str.contains('jpeg') |
                                       train_funny.prep.str.contains('http') |

                                       train_funny.prep.str.contains('https') |

                                       train_funny.prep.str.contains('picture') |
                                       train_funny.prep.str.contains('gif') |
                                       train_funny.prep.str.contains('lmao') |
                                       
                                       
                                       train_funny.prep.str.contains('lol') |
                                       
                                       train_funny.prep.str.contains('omg') |
                                       
                                       train_funny.prep.str.contains('haha') |
                                       
                                       
                                       
                                       train_funny.comment.str.contains('!!') |
                                       train_funny.prep.str.contains('laugh') |
                                       
                                      

                                       train_funny.prep.str.contains('num') 
                                       
                                


                                      ),'is funny', '.') 

train_funny



/usr/lib/python3/dist-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,comment,subreddit,prep,new_comment
9,9,https://i.imgur.com/DUdy0KL.jpg,funny,httpsiimgurcomdudy num kljpg,is funny
23,23,All music can broken down to structures. Point...,funny,music broken structure pointing structure kind...,.
24,24,I too bring my bow into the hospital after I’v...,funny,bring bow hospital ive shot dad shoulder arrow...,.
45,45,Go to the gym and eat a healthy diet. \n\nAnd ...,funny,go gym eat healthy diet get sunlight,.
49,49,"Politicians too. Willing to roast each other, ...",funny,politician willing roast oh shit dont sound an...,.
...,...,...,...,...,...
59883,59883,"Maybe hold your reaction till you get facts, h...",funny,maybe hold reaction till get fact,.
59906,59906,I would've taken my gun out and shot that ball...,funny,wouldve taken gun shot balloon wa making escap...,.
59915,59915,I most certainly would. 4 backhoes sitting in ...,funny,certainly would num backhoe sitting yard komat...,is funny
59983,59983,The only time colon broke that I can easily re...,funny,time colon broke easily recall watching animal...,.


In [10]:

train_conspiracy=train.loc[train['subreddit']=='conspiracy']


train_conspiracy['new_comment'] = np.where((train_conspiracy.prep.str.contains('how')|
                                     train_conspiracy.prep.str.contains('since') |
                                     train_conspiracy.prep.str.contains('http') |
                                     train_conspiracy.prep.str.contains('https') |                                   
                                     train_conspiracy.prep.str.contains('what') |
                                            
                                       train_conspiracy.prep.str.contains('jpg')|
                                       train_conspiracy.prep.str.contains('jpeg') |
                                       train_conspiracy.prep.str.contains('http') |

                                       train_conspiracy.prep.str.contains('https') |

                                       train_conspiracy.prep.str.contains('picture') |
                                       train_conspiracy.prep.str.contains('gif') |

                           

                                       train_conspiracy.prep.str.contains('num')
                          


                                      ),'is conpiracy', '.') 

train_conspiracy



/usr/lib/python3/dist-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,id,comment,subreddit,prep,new_comment
11,11,Is it really that far fetched? It actually mak...,conspiracy,really far fetched actually make perfect sense...,is conpiracy
14,14,Do you believe in ancestral spirit? Maybe this...,conspiracy,believe ancestral spirit maybe synchronicity h...,.
15,15,The JFK assassination.,conspiracy,jfk assassination,.
47,47,"Yes it was. Which I always found confusing, co...",conspiracy,yes wa always found confusing considering wore...,.
78,78,As the story goes they were waiting for Muelle...,conspiracy,story go waiting mueller wrap thats also fishy...,is conpiracy
...,...,...,...,...,...
59931,59931,Those Twitter comments smh,conspiracy,twitter comment smh,.
59937,59937,I practice meditation. It has really helped me...,conspiracy,practice meditation ha really helped control a...,.
59942,59942,"[For the First Time Ever, Temperatures Reached...",conspiracy,first time ever temperature reached num degree...,is conpiracy
59970,59970,Well... scooter guy. 80s. Vancouver BC. Down t...,conspiracy,well scooter guy num vancouver bc num u,is conpiracy


In [11]:
df_feat_eng=train_funny.append(train_Ask)
df_feat_eng

,id,comment,subreddit,prep,new_comment
9,9,https://i.imgur.com/DUdy0KL.jpg,funny,httpsiimgurcomdudy num kljpg,is funny
23,23,All music can broken down to structures. Point...,funny,music broken structure pointing structure kind...,.
24,24,I too bring my bow into the hospital after I’v...,funny,bring bow hospital ive shot dad shoulder arrow...,.
45,45,Go to the gym and eat a healthy diet. \n\nAnd ...,funny,go gym eat healthy diet get sunlight,.
49,49,"Politicians too. Willing to roast each other, ...",funny,politician willing roast oh shit dont sound an...,.
...,...,...,...,...,...
59891,59891,Breaking bad. Definitely.,AskReddit,breaking bad definitely,.
59899,59899,There's a lot of articles out there. Gotta sea...,AskReddit,lot article gotta search engine one although a...,is AskReddit
59990,59990,Children are still dying from preventable dise...,AskReddit,child still dying preventable disease around w...,.
59993,59993,What's a good costume for a muscular girl with...,AskReddit,whats good costume muscular girl visible ab,is AskReddit


In [12]:
df_feat_eng=df_feat_eng.append(train_conspiracy)
df_feat_eng

,id,comment,subreddit,prep,new_comment
9,9,https://i.imgur.com/DUdy0KL.jpg,funny,httpsiimgurcomdudy num kljpg,is funny
23,23,All music can broken down to structures. Point...,funny,music broken structure pointing structure kind...,.
24,24,I too bring my bow into the hospital after I’v...,funny,bring bow hospital ive shot dad shoulder arrow...,.
45,45,Go to the gym and eat a healthy diet. \n\nAnd ...,funny,go gym eat healthy diet get sunlight,.
49,49,"Politicians too. Willing to roast each other, ...",funny,politician willing roast oh shit dont sound an...,.
...,...,...,...,...,...
59931,59931,Those Twitter comments smh,conspiracy,twitter comment smh,.
59937,59937,I practice meditation. It has really helped me...,conspiracy,practice meditation ha really helped control a...,.
59942,59942,"[For the First Time Ever, Temperatures Reached...",conspiracy,first time ever temperature reached num degree...,is conpiracy
59970,59970,Well... scooter guy. 80s. Vancouver BC. Down t...,conspiracy,well scooter guy num vancouver bc num u,is conpiracy


In [13]:
train['new_comment']= df_feat_eng['new_comment']






train.new_comment=train.new_comment.fillna('')
train



,id,comment,subreddit,prep,new_comment
0,0,I think prestige points should not expire ever...,leagueoflegends,think prestige point expire ever skin buy avai...,
1,1,Whats going to happen with them if they will b...,europe,whats going happen refused asilum appeal,
2,2,Anecdotal evidence is anecdotal. Clearly by “e...,gameofthrones,anecdotal evidence anecdotal clearly everyone ...,
3,3,"Look dude, with all due respect, your music is...",Music,look dude due respect music isnt people look l...,
4,4,Hope he gets the doomhammer back!,wow,hope get doomhammer back,
...,...,...,...,...,...
59995,59995,Yo this guy Luka pretty good,nba,yo guy luka pretty good,
59996,59996,Unplug these things right now,funny,unplug thing right,.
59997,59997,Well said. Do you think they’ll resonate with ...,movies,well said think theyll resonate future generat...,
59998,59998,So we can impeach a president for lying? Pleas...,worldnews,impeach president lying please inform trump th...,


In [14]:
train['new_comment2']=train['comment']+'-'+train['new_comment']
train[0:50]

,id,comment,subreddit,prep,new_comment,new_comment2
0,0,I think prestige points should not expire ever...,leagueoflegends,think prestige point expire ever skin buy avai...,,I think prestige points should not expire ever...
1,1,Whats going to happen with them if they will b...,europe,whats going happen refused asilum appeal,,Whats going to happen with them if they will b...
2,2,Anecdotal evidence is anecdotal. Clearly by “e...,gameofthrones,anecdotal evidence anecdotal clearly everyone ...,,Anecdotal evidence is anecdotal. Clearly by “e...
3,3,"Look dude, with all due respect, your music is...",Music,look dude due respect music isnt people look l...,,"Look dude, with all due respect, your music is..."
4,4,Hope he gets the doomhammer back!,wow,hope get doomhammer back,,Hope he gets the doomhammer back!-
5,5,Trading for coaches has happened before,nfl,trading coach ha happened,,Trading for coaches has happened before-
6,6,"Considering what the kid has already seen, did...",movies,considering kid ha already seen didnt figure w...,,"Considering what the kid has already seen, did..."
7,7,Nah clearly it's Tom Bombadil,movies,nah clearly tom bombadil,,Nah clearly it's Tom Bombadil-
8,8,Time to go play some Elite Dangerous in VR I t...,Music,time go play elite dangerous vr think thanks,,Time to go play some Elite Dangerous in VR I t...
9,9,https://i.imgur.com/DUdy0KL.jpg,funny,httpsiimgurcomdudy num kljpg,is funny,https://i.imgur.com/DUdy0KL.jpg-is funny


In [89]:

features=train['new_comment2']
target = train['subreddit']
features

0        I think prestige points should not expire ever...
1        Whats going to happen with them if they will b...
2        Anecdotal evidence is anecdotal. Clearly by “e...
3        Look dude, with all due respect, your music is...
4                       Hope he gets the doomhammer back!-
                               ...                        
59995                        Yo this guy Luka pretty good-
59996                      Unplug these things right now-.
59997    Well said. Do you think they’ll resonate with ...
59998    So we can impeach a president for lying? Pleas...
59999    Too broad dude, get ready for the shit my pant...
Name: new_comment2, Length: 60000, dtype: object

In [37]:
train['new_comment2_prep']=train['prep']+'-'+train['new_comment']
features_prep=train['new_comment2_prep']
target = train['subreddit']
train

,id,comment,subreddit,prep,new_comment,new_comment2,new_comment2_prep
0,0,I think prestige points should not expire ever...,leagueoflegends,think prestige point expire ever skin buy avai...,,I think prestige points should not expire ever...,think prestige point expire ever skin buy avai...
1,1,Whats going to happen with them if they will b...,europe,whats going happen refused asilum appeal,,Whats going to happen with them if they will b...,whats going happen refused asilum appeal-
2,2,Anecdotal evidence is anecdotal. Clearly by “e...,gameofthrones,anecdotal evidence anecdotal clearly everyone ...,,Anecdotal evidence is anecdotal. Clearly by “e...,anecdotal evidence anecdotal clearly everyone ...
3,3,"Look dude, with all due respect, your music is...",Music,look dude due respect music isnt people look l...,,"Look dude, with all due respect, your music is...",look dude due respect music isnt people look l...
4,4,Hope he gets the doomhammer back!,wow,hope get doomhammer back,,Hope he gets the doomhammer back!-,hope get doomhammer back-
...,...,...,...,...,...,...,...
59995,59995,Yo this guy Luka pretty good,nba,yo guy luka pretty good,,Yo this guy Luka pretty good-,yo guy luka pretty good-
59996,59996,Unplug these things right now,funny,unplug thing right,.,Unplug these things right now-.,unplug thing right-.
59997,59997,Well said. Do you think they’ll resonate with ...,movies,well said think theyll resonate future generat...,,Well said. Do you think they’ll resonate with ...,well said think theyll resonate future generat...
59998,59998,So we can impeach a president for lying? Pleas...,worldnews,impeach president lying please inform trump th...,,So we can impeach a president for lying? Pleas...,impeach president lying please inform trump th...


In [88]:
features

<60000x58896 sparse matrix of type '<class 'numpy.float64'>'
	with 1367111 stored elements in Compressed Sparse Row format>

In [45]:
#LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import Normalizer



from textblob import TextBlob
from nltk.stem.porter import PorterStemmer
import re
porter_stemmer = PorterStemmer()

# Try TextBlob
def textblob_tokenizer(str_input):
    blob = TextBlob(str_input.lower())
    tokens = blob.words
    words = [token.stem() for token in tokens]
    return words

# Try NLTK's PorterStemmer
def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words


from textblob import TextBlob
from nltk.stem.porter import PorterStemmer



param_dist = { 'clf__penalty': ['l2'],
              'clf__loss' : ['hinge', 'squared_hinge'],
              'clf__C' : np.linspace(0.1,2,5),}


pipeline_LinearSVC = Pipeline([
                    #('vect', CountVectorizer(tokenizer = stemming_tokenizer)),

                     ('vect', CountVectorizer()),#tokenizer = textblob_tokenizer)),#ngram_range=(1,2), min_df = 10, max_df = 1.)),
                     ('tfidf', TfidfTransformer()),
                     ('norm', Normalizer()),
                     ('clf', LinearSVC(C=1.0, penalty='l2', max_iter=3000, dual=False,random_state=0))#LinearSVC(random_state=0))
])

  #  ('vect', feature_extraction.text.CountVectorizer(tokenizer = textblob_tokenizer))
    #


grid_search = RandomizedSearchCV(pipeline_LinearSVC, param_dist, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')
grid_search.fit(features, target)

import time

start_time = time.time()

              
print(grid_search.best_score_)
print(grid_search.cv_results_)

#previsoes_test_pre_proc=grid_search.predict(test['comment'])

#0.4337


#com mew_comment2_prep sem grid 0.4720833333333333


#com mew_comment2_prep com grid 0.0.48285

#feature e grid 0.5216

#feature_prep e grid 0.5206166666666667

#feature_prep e grid l1 l2 


#0.5359833333333334
#0.5359833333333334

#bruto cv3 0.5495833333333334

#prep cv3 0.5488333333333334



#bruto cv10 0.56965

#prep cv10 0.5662833333333332

#bruto cv5 0.5638666666666667

#prep cv5 0.5609333333333334


#muitas regras = overfit

#bruto  textblob cv5 0.6045833333333335

#bruto steeming cv5 0.5988166666666667

#('vect', CountVectorizer(tokenizer = textblob_tokenizer,ngram_range=(1,2), min_df = 10, max_df = 1.)), = >0.5636


 

#com menos regras para funny conspiracy e AskReddit e textblob    0.52955

#com menos regras para funny conspiracy e AskReddit e features_prep 0.5208166666666667


#com menos regras para funny conspiracy e AskReddit e textblob features_prep 0.5234666666666665 

#com menos regras para funny conspiracy e AskReddit  features_prep  0.5208166666666667


#com menos regras para funny conspiracy e AskReddit  features    0.5286500000000001


#text blob nao vale a pena


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  7.6min finished


0.5286500000000001
{'mean_fit_time': array([ 8.68239546, 63.82216992,  8.21497512, 71.07705617,  7.30583596,
       65.04690614,  6.76906452, 60.44162979,  6.34766388, 43.6112803 ]), 'std_fit_time': array([ 0.48121734,  2.26974939,  0.34644865,  3.11993462,  0.7473942 ,
        3.51532505,  0.30797966,  6.6746805 ,  0.65427454, 12.48336149]), 'mean_score_time': array([0.        , 2.1289463 , 0.        , 1.67688828, 0.        ,
       1.48496294, 0.        , 1.47402387, 0.        , 0.59752498]), 'std_score_time': array([0.        , 0.14758079, 0.        , 0.13349226, 0.        ,
       0.18959255, 0.        , 0.21922945, 0.        , 0.1612728 ]), 'param_clf__penalty': masked_array(data=['l2', 'l2', 'l2', 'l2', 'l2', 'l2', 'l2', 'l2', 'l2',
                   'l2'],
             mask=[False, False, False, False, False, False, False, False,
                   False, False],
       fill_value='?',
            dtype=object), 'param_clf__loss': masked_array(data=['hinge', 'squared_hinge', 'h

##### features_prep vai com test_prep

features vai com test

In [66]:
def report(results, top=3):
    for i in range(1, top+1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


#Logistic Regression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import Normalizer
norm=Normalizer()
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import time


param_dist = {'clf__penalty': ['l2'],
              'clf__solver': ['liblinear'],
              'clf__C': np.linspace(0.1,5,5)}
clf = LogisticRegression(multi_class='auto')
text_clf = Pipeline([('vect',CountVectorizer(stop_words='english', ngram_range=(1,2), min_df = 10, max_df = 1.)),('tfidf',TfidfTransformer()),('clf',clf)])
random_search = GridSearchCV(text_clf, param_grid = param_dist, cv=10)
start_time = time.time()
random_search.fit(features, target)
print("GridSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time.time() - start_time), len(random_search.cv_results_['params'])))
report(random_search.cv_results_)

GridSearchCV took 1124.24 seconds for 5 candidates parameter settings.


NameError: name 'report' is not defined

In [67]:
def report(results, top=3):
    for i in range(1, top+1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [68]:
report(random_search.cv_results_)

Model with rank: 1
Mean validation score: 0.487 (std: 0.008)
Parameters: {'clf__C': 1.3250000000000002, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}

Model with rank: 2
Mean validation score: 0.485 (std: 0.007)
Parameters: {'clf__C': 2.5500000000000003, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}

Model with rank: 3
Mean validation score: 0.483 (std: 0.007)
Parameters: {'clf__C': 3.7750000000000004, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}



In [69]:
features[0:10]

0    I think prestige points should not expire ever...
1    Whats going to happen with them if they will b...
2    Anecdotal evidence is anecdotal. Clearly by “e...
3    Look dude, with all due respect, your music is...
4                   Hope he gets the doomhammer back!-
5             Trading for coaches has happened before-
6    Considering what the kid has already seen, did...
7                       Nah clearly it's Tom Bombadil-
8    Time to go play some Elite Dangerous in VR I t...
9             https://i.imgur.com/DUdy0KL.jpg-is funny
Name: new_comment2, dtype: object

### Bayesian Optimization

In [70]:
from skopt import gp_minimize

In [85]:
#LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import Normalizer
from textblob import TextBlob
from nltk.stem.porter import PorterStemmer
import re
from textblob import TextBlob
from nltk.stem.porter import PorterStemmer


pipeline_LinearSVC = Pipeline([
                    #('vect', CountVectorizer(tokenizer = stemming_tokenizer)),

                     ('vect', CountVectorizer()),#tokenizer = textblob_tokenizer)),#ngram_range=(1,2), min_df = 10, max_df = 1.)),
                     ('tfidf', TfidfTransformer()),
                     ('norm', Normalizer()),
                     ('clf', LinearSVC(dual=False,random_state=0))#LinearSVC(random_state=0))
])

  #  ('vect', feature_extraction.text.CountVectorizer(tokenizer = textblob_tokenizer))
    #


#grid_search = RandomizedSearchCV(pipeline_LinearSVC,{}, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')
#grid_search.fit(features, target)

import time

start_time = time.time()

              
#print(grid_search.best_score_)
#print(grid_search.cv_results_)

In [91]:
from sklearn.model_selection import train_test_split
previsores_train, previsores_valid, classe_train, classe_valid = train_test_split(features, target, test_size=0.20, random_state=0)



In [92]:
# pipeline e parametros
def treinar_modelo (params):
    penalty=params[0]
    loss=params[1]
    tol=params[2]
    c=params[3]
    multi_class=params[4]
    max_iter=params[5]
    
    
    print(params, '\n')
    
    
    
    mdl = pipeline_LinearSVC
    mdl.fit(previsores_train,classe_train)
    
    p = mdl.predict(previsores_valid)
    
    return accuracy_score(classe_valid,p)
space=[
       ('l1','l2'),#penalty
       ('hinge','squared_hinge'),#loss
       (1e-7,1e-1),#tol
       (0.1,1.0),#c
       ('ovr','crammer_singer'),#multi_class
       (1000,5000)#max_iter
       ]

resultado = dummy_minimize(treinar_modelo,space,random_state=1,verbose=1,n_calls=30)


Iteration No: 1 started. Evaluating function at random point.
['l2', 'squared_hinge', 0.012812531980490881, 0.9991364637917304, 'ovr', 1129] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 28.3240
Function value obtained: 0.5284
Current minimum: 0.5284
Iteration No: 2 started. Evaluating function at random point.
['l1', 'hinge', 0.03967680774631958, 0.5849350606030214, 'ovr', 3962] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 30.2748
Function value obtained: 0.5284
Current minimum: 0.5284
Iteration No: 3 started. Evaluating function at random point.
['l2', 'hinge', 0.022957798415261185, 0.5809725180523154, 'crammer_singer', 4049] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 28.5527
Function value obtained: 0.5284
Current minimum: 0.5284
Iteration No: 4 started. Evaluating function at random point.
['l1', 'squared_hinge', 0.07783894579472973, 0.7443734643715278, 'crammer_singer', 3877] 

Iteration No: 4 ended. Evalu

In [93]:
gp_minimize(treinar_modelo,space,random_state=1,verbose=1,n_calls=30,n_random_starts=10)

Iteration No: 1 started. Evaluating function at random point.
['l2', 'squared_hinge', 0.012812531980490883, 0.9991364637917305, 'ovr', 2586] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 25.3869
Function value obtained: 0.5284
Current minimum: 0.5284
Iteration No: 2 started. Evaluating function at random point.
['l1', 'squared_hinge', 0.09355391352669613, 0.8616798250174156, 'ovr', 3098] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 26.5287
Function value obtained: 0.5284
Current minimum: 0.5284
Iteration No: 3 started. Evaluating function at random point.
['l1', 'hinge', 0.05344143745331082, 0.9225658221213098, 'ovr', 2723] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 31.5814
Function value obtained: 0.5284
Current minimum: 0.5284
Iteration No: 4 started. Evaluating function at random point.
['l2', 'squared_hinge', 0.07159707999978485, 0.8224817535436286, 'ovr', 3073] 

Iteration No: 4 ended. Evaluation done at r

ValueError: array must not contain infs or NaNs

In [62]:
predict_test=grid_search.predict(test['comment'])


In [63]:
predict_test

array(['hockey', 'europe', 'funny', ..., 'trees', 'movies',
       'GlobalOffensive'], dtype=object)

In [64]:

predict_test = pd.DataFrame(predict_test, columns=['Category'])

predict_test['Id'] = predict_test.index

predict_test = predict_test.reindex(columns=['Id', 'Category'])

predict_test.to_csv('SVM_mandar_essa_60_test_brut_textblob.csv', index=False)

In [47]:

features=train['new_comment2']
target = train['subreddit']

In [95]:
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.preprocessing import Normalizer
norm=Normalizer()
from sklearn.preprocessing import StandardScaler
vect=CountVectorizer()
tfidf=TfidfTransformer()


features=vect.fit_transform(features)

features=tfidf.fit_transform(features)

features=norm.fit_transform(features)






In [96]:
from sklearn.model_selection import train_test_split
previsores_train, previsores_valid, classe_train, classe_valid = train_test_split(features, target, test_size=0.20, random_state=0)



In [107]:
#sem PIPE e COm Dummy



# pipeline e parametros
def treinar_modelo (params):
    penalty=params[0]
    loss=params[1]
    tol=params[2]
    c=params[3]
    multi_class=params[4]
    max_iter=params[5]
    
    
    print(params, '\n')
    
    
    
    mdl =LinearSVC(C=c, max_iter=max_iter, loss=loss, tol=tol, multi_class=multi_class,random_state=0)
    mdl.fit(previsores_train,classe_train)
    
    p = mdl.predict(previsores_valid)
    
    return accuracy_score(classe_valid,p)
space=[
       ('l1','l2'),#penalty
       ('hinge','squared_hinge'),#loss
       (1e-7,1e-1),#tol
       (0.1,1.0),#c
       ('ovr','crammer_singer'),#multi_class
       (1000,5000)#max_iter
       ]

resultado = dummy_minimize(treinar_modelo,space,random_state=1,verbose=1,n_calls=30)


Iteration No: 1 started. Evaluating function at random point.
['l2', 'squared_hinge', 0.012812531980490881, 0.9991364637917304, 'ovr', 1129] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 4.6117
Function value obtained: 0.5282
Current minimum: 0.5282
Iteration No: 2 started. Evaluating function at random point.
['l1', 'hinge', 0.03967680774631958, 0.5849350606030214, 'ovr', 3962] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 4.0543
Function value obtained: 0.5355
Current minimum: 0.5282
Iteration No: 3 started. Evaluating function at random point.
['l2', 'hinge', 0.022957798415261185, 0.5809725180523154, 'crammer_singer', 4049] 



/home/gabriel/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 2.8683
Function value obtained: 0.5367
Current minimum: 0.5282
Iteration No: 4 started. Evaluating function at random point.
['l1', 'squared_hinge', 0.07783894579472973, 0.7443734643715278, 'crammer_singer', 3877] 

Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.0096
Function value obtained: 0.5362
Current minimum: 0.5282
Iteration No: 5 started. Evaluating function at random point.
['l2', 'squared_hinge', 0.0894606768897184, 0.17653979023280014, 'ovr', 1431] 

Iteration No: 5 ended. Evaluation done at random point.
Time taken: 2.1970
Function value obtained: 0.5350
Current minimum: 0.5282
Iteration No: 6 started. Evaluating function at random point.
['l1', 'squared_hinge', 0.05930659252325387, 0.704488687679921, 'ovr', 1515] 

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 3.5832
Function value obtained: 0.5321
Current minimum: 0.5282
Iteration No: 7 started. Evaluating func

In [108]:
gp_minimize(treinar_modelo,space,random_state=1,verbose=1,n_calls=30,n_random_starts=10)

Iteration No: 1 started. Evaluating function at random point.
['l2', 'squared_hinge', 0.012812531980490883, 0.9991364637917305, 'ovr', 2586] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 4.4071
Function value obtained: 0.5282
Current minimum: 0.5282
Iteration No: 2 started. Evaluating function at random point.
['l1', 'squared_hinge', 0.09355391352669613, 0.8616798250174156, 'ovr', 3098] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 3.3423
Function value obtained: 0.5293
Current minimum: 0.5282
Iteration No: 3 started. Evaluating function at random point.
['l1', 'hinge', 0.05344143745331082, 0.9225658221213098, 'ovr', 2723] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 4.2376
Function value obtained: 0.5381
Current minimum: 0.5282
Iteration No: 4 started. Evaluating function at random point.
['l2', 'squared_hinge', 0.07159707999978485, 0.8224817535436286, 'ovr', 3073] 

Iteration No: 4 ended. Evaluation done at rand

/home/gabriel/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iteration No: 7 ended. Evaluation done at random point.
Time taken: 11.4691
Function value obtained: 0.5357
Current minimum: 0.5282
Iteration No: 8 started. Evaluating function at random point.
['l1', 'hinge', 0.0073365098378427845, 0.5223146737943289, 'ovr', 4613] 

Iteration No: 8 ended. Evaluation done at random point.
Time taken: 6.3150
Function value obtained: 0.5367
Current minimum: 0.5282
Iteration No: 9 started. Evaluating function at random point.
['l1', 'squared_hinge', 0.008362392092402167, 0.9251752010767559, 'crammer_singer', 2196] 

Iteration No: 9 ended. Evaluation done at random point.
Time taken: 6.4989
Function value obtained: 0.5314
Current minimum: 0.5282
Iteration No: 10 started. Evaluating function at random point.
['l2', 'squared_hinge', 0.061393870385773805, 0.9608820910034702, 'ovr', 1924] 

Iteration No: 10 ended. Evaluation done at random point.
Time taken: 4.3803
Function value obtained: 0.5288
Current minimum: 0.5282
Iteration No: 11 started. Searching for 

/home/gabriel/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iteration No: 13 ended. Search finished for the next optimal point.
Time taken: 101.6508
Function value obtained: 0.5337
Current minimum: 0.5281
Iteration No: 14 started. Searching for the next optimal point.
['l1', 'squared_hinge', 1e-07, 1.0, 'ovr', 5000] 



...

/home/gabriel/.local/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 29 ended. Search finished for the next optimal point.
Time taken: 11.0118
Function value obtained: 0.5281
Current minimum: 0.5281
Iteration No: 30 started. Searching for the next optimal point.
['l1', 'squared_hinge', 1e-07, 1.0, 'ovr', 5000] 



/home/gabriel/.local/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 11.5757
Function value obtained: 0.5281
Current minimum: 0.5281


          fun: 0.5280833333333333
    func_vals: array([0.52816667, 0.52933333, 0.53808333, 0.53066667, 0.53475   ,
       0.53416667, 0.53575   , 0.53666667, 0.53141667, 0.52875   ,
       0.52808333, 0.5285    , 0.53375   , 0.52808333, 0.52808333,
       0.52808333, 0.52808333, 0.52808333, 0.52808333, 0.52808333,
       0.52808333, 0.52808333, 0.52808333, 0.52808333, 0.52808333,
       0.52808333, 0.52808333, 0.52808333, 0.52808333, 0.52808333])
       models: [GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 1, 1, 1, 1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=2, noise='gaussian',
                         normalize_y=True, random_state=1791095845), GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 1, 1, 1, 1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                         n_restarts_optimizer=2, noise='gaussian',
                         normalize_y=True, random_state=1791095845), GaussianProcessRegressor(k

In [109]:
gp_minimize(treinar_modelo,space,random_state=1,verbose=1,n_calls=100,n_random_starts=20)

Iteration No: 1 started. Evaluating function at random point.
['l2', 'squared_hinge', 0.012812531980490883, 0.9991364637917305, 'ovr', 2586] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 4.8426
Function value obtained: 0.5282
Current minimum: 0.5282
Iteration No: 2 started. Evaluating function at random point.
['l1', 'squared_hinge', 0.09355391352669613, 0.8616798250174156, 'ovr', 3098] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 3.8887
Function value obtained: 0.5293
Current minimum: 0.5282
Iteration No: 3 started. Evaluating function at random point.
['l1', 'hinge', 0.05344143745331082, 0.9225658221213098, 'ovr', 2723] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 4.6645
Function value obtained: 0.5381
Current minimum: 0.5282
Iteration No: 4 started. Evaluating function at random point.
['l2', 'squared_hinge', 0.07159707999978485, 0.8224817535436286, 'ovr', 3073] 

Iteration No: 4 ended. Evaluation done at rand

/home/gabriel/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iteration No: 7 ended. Evaluation done at random point.
Time taken: 11.2852
Function value obtained: 0.5357
Current minimum: 0.5282
Iteration No: 8 started. Evaluating function at random point.
['l1', 'hinge', 0.0073365098378427845, 0.5223146737943289, 'ovr', 4613] 

Iteration No: 8 ended. Evaluation done at random point.
Time taken: 6.9565
Function value obtained: 0.5367
Current minimum: 0.5282
Iteration No: 9 started. Evaluating function at random point.
['l1', 'squared_hinge', 0.008362392092402167, 0.9251752010767559, 'crammer_singer', 2196] 

Iteration No: 9 ended. Evaluation done at random point.
Time taken: 6.4379
Function value obtained: 0.5314
Current minimum: 0.5282
Iteration No: 10 started. Evaluating function at random point.
['l2', 'squared_hinge', 0.061393870385773805, 0.9608820910034702, 'ovr', 1924] 

Iteration No: 10 ended. Evaluation done at random point.
Time taken: 3.9415
Function value obtained: 0.5288
Current minimum: 0.5282
Iteration No: 11 started. Evaluating fun

/home/gabriel/.local/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 23 ended. Search finished for the next optimal point.
Time taken: 11.4335
Function value obtained: 0.5281
Current minimum: 0.5281
Iteration No: 24 started. Searching for the next optimal point.
['l1', 'squared_hinge', 0.035340694079044124, 1.0, 'ovr', 3309] 

Iteration No: 24 ended. Search finished for the next optimal point.
Time taken: 5.7096
Function value obtained: 0.5278
Current minimum: 0.5278
Iteration No: 25 started. Searching for the next optimal point.
['l1', 'squared_hinge', 1e-07, 1.0, 'ovr', 3394] 

Iteration No: 25 ended. Search finished for the next optimal point.
Time taken: 11.2345
Function value obtained: 0.5281
Current minimum: 0.5278
Iteration No: 26 started. Searching for the next optimal point.
['l1', 'squared_hinge', 0.1, 1.0, 'ovr', 4064] 

Iteration No: 26 ended. Search finished for the next optimal point.
Time taken: 5.9800
Function value obtained: 0.5285
Current minimum: 0.5278
Iteration No: 27 started. Searching for the next optimal point.
['l1

...

/home/gabriel/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 6.4057
Function value obtained: 0.5319
Current minimum: 0.5278
Iteration No: 65 started. Searching for the next optimal point.
['l1', 'squared_hinge', 0.03129430983404321, 1.0, 'ovr', 4407] 

Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 7.1042
Function value obtained: 0.5278
Current minimum: 0.5278
Iteration No: 66 started. Searching for the next optimal point.
['l1', 'squared_hinge', 0.03129354026443392, 1.0, 'ovr', 3532] 

Iteration No: 66 ended. Search finished for the next optimal point.
Time taken: 7.1979
Function value obtained: 0.5278
Current minimum: 0.5278
Iteration No: 67 started. Searching for the next optimal point.
['l1', 'squared_hinge', 0.03129549085671713, 1.0, 'ovr', 5000] 

Iteration No: 67 ended. Search finished for the next optimal point.
Time taken: 7.6029
Function value obtained: 0.5278
Current minimum: 0.5278
Iteration No: 68 started. Searching for t

/home/gabriel/.local/lib/python3.6/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Iteration No: 86 ended. Search finished for the next optimal point.
Time taken: 3.6247
Function value obtained: 0.5376
Current minimum: 0.5278
Iteration No: 87 started. Searching for the next optimal point.
['l2', 'squared_hinge', 0.06586760761571718, 0.9828247126403987, 'ovr', 1759] 

Iteration No: 87 ended. Search finished for the next optimal point.
Time taken: 6.8099
Function value obtained: 0.5281
Current minimum: 0.5278
Iteration No: 88 started. Searching for the next optimal point.
['l1', 'squared_hinge', 0.01812310106931588, 0.6406429154593035, 'ovr', 1244] 

Iteration No: 88 ended. Search finished for the next optimal point.
Time taken: 6.7751
Function value obtained: 0.5327
Current minimum: 0.5278
Iteration No: 89 started. Searching for the next optimal point.
['l1', 'squared_hinge', 0.024266128213620196, 1.0, 'ovr', 2487] 

Iteration No: 89 ended. Search finished for the next optimal point.
Time taken: 7.5973
Function value obtained: 0.5281
Current minimum: 0.5278
Iteration 

          fun: 0.5278333333333334
    func_vals: array([0.52816667, 0.52933333, 0.53808333, 0.53066667, 0.53475   ,
       0.53416667, 0.53575   , 0.53666667, 0.53141667, 0.52875   ,
       0.53683333, 0.531     , 0.53566667, 0.53625   , 0.53191667,
       0.53766667, 0.53683333, 0.5355    , 0.53658333, 0.537     ,
       0.52808333, 0.5285    , 0.52808333, 0.52783333, 0.52808333,
       0.5285    , 0.52791667, 0.528     , 0.52808333, 0.52825   ,
       0.52808333, 0.52816667, 0.52808333, 0.52816667, 0.52808333,
       0.52825   , 0.52808333, 0.52808333, 0.52825   , 0.5285    ,
       0.52808333, 0.52808333, 0.52808333, 0.52808333, 0.52783333,
       0.52841667, 0.52808333, 0.52808333, 0.52808333, 0.52783333,
       0.52808333, 0.52783333, 0.52783333, 0.52783333, 0.52783333,
       0.52783333, 0.52783333, 0.52783333, 0.52783333, 0.52825   ,
       0.52783333, 0.52783333, 0.52783333, 0.53191667, 0.52783333,
       0.52783333, 0.52783333, 0.52816667, 0.52783333, 0.53275   ,
       0.5278

In [50]:
#classifier


clf=LinearSVC(C=1.0, penalty='l2', max_iter=3000, dual=False,random_state=0)

clf.fit(previsores_train,classe_train)

label_predict=clf.predict(previsores_valid)




In [51]:
acuracia=accuracy_score(classe_valid,label_predict)
print('acuracia: ',acuracia)

acuracia:  0.5281666666666667


In [ ]:
'''acuracia:  0.49725
    
    0.4856666666666667
    
    0.4984166666666667
    
    acuracia:  0.5029166666666667
    
    acuracia:  0.4900833333333333
'''
acuracia:  0.5216666666666666
acuracia:  0.52175 sem normalizer
acuracia:  0.5400833333333334 sem norm funny + conspiracy
acuracia:  0.5400833333333334 com norm funny + conspiracy

0.5740833333333333 prep conp + ask+ funny

acuracia:  0.56825 prep conp + ask+ funny
    
    
com menos coisas nas 3 categorias acuracia:  0.5281666666666667




test_data

In [ ]:
features_test=vect.fit_transform(test['comment'])

features_test=tfidf.fit_transform(features_test)

features_test=norm.fit_transform(features_test)




In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
features_test1=csr_matrix(features_test,(20000, 58895)).toarray()


In [ ]:

#features_test
previsoes_test=clf.predict(features_test1)
previsoes_test

In [ ]:

previsoes_test = pd.DataFrame(previsoes_test, columns=['Category'])

previsoes_test['Id'] = previsoes_test.index

previsoes_test = previsoes_test.reindex(columns=['Id', 'Category'])

previsoes_test.to_csv('linear_svc_20jul.csv', index=False)




In [52]:
classe_valid2 = pd.DataFrame(classe_valid)




classe_valid2['previsao']=label_predict
classe_valid2

,subreddit,previsao
3048,hockey,hockey
19563,movies,movies
58303,baseball,baseball
8870,europe,europe
40228,Overwatch,Overwatch
...,...,...
47049,nfl,nfl
31629,trees,funny
50430,anime,anime
21574,baseball,baseball


In [53]:
classe_valid2.describe()

,subreddit,previsao
count,12000,12000
unique,20,20
top,Music,trees
freq,630,745


In [54]:
classe_valid = pd.DataFrame(classe_valid,columns=['subreddit'])
classe_valid
classe_valid.groupby(['subreddit']).count()

""
subreddit
AskReddit
GlobalOffensive
Music
Overwatch
anime
baseball
canada
conspiracy
europe


In [55]:
classe_valid['1']=1
classe_valid.groupby(['subreddit']).count()

,1
subreddit,
AskReddit,590
GlobalOffensive,592
Music,630
Overwatch,608
anime,608
baseball,601
canada,588
conspiracy,541
europe,595


In [56]:
classe_valid2['Id'] = classe_valid2.index
classe_valid2


,subreddit,previsao,Id
3048,hockey,hockey,3048
19563,movies,movies,19563
58303,baseball,baseball,58303
8870,europe,europe,8870
40228,Overwatch,Overwatch,40228
...,...,...,...
47049,nfl,nfl,47049
31629,trees,funny,31629
50430,anime,anime,50430
21574,baseball,baseball,21574


In [57]:
classe_valid2['igual']= np.where(classe_valid2['previsao']==classe_valid2['subreddit'],1,0)

In [58]:
acertos=classe_valid2.groupby(['subreddit']).sum()
acertos.sort_values(by='igual')

,Id,igual
subreddit,,
funny,18543759,186
AskReddit,17636754,214
worldnews,17133584,228
conspiracy,16230416,254
canada,17330563,277
europe,17760953,296
Music,19110665,307
gameofthrones,17883785,308
movies,17244947,324


In [59]:
count=classe_valid.groupby(['subreddit']).count()
count.sort_values(by='subreddit')

,1
subreddit,
AskReddit,590
GlobalOffensive,592
Music,630
Overwatch,608
anime,608
baseball,601
canada,588
conspiracy,541
europe,595


In [60]:

acertos['count_subreddit']=count['1']
acertos['%_acertos']=acertos['igual']/acertos['count_subreddit']

acertos.sort_values(by='%_acertos',ascending=True)

,Id,igual,count_subreddit,%_acertos
subreddit,,,,
funny,18543759,186,609,0.305419
AskReddit,17636754,214,590,0.362712
worldnews,17133584,228,590,0.386441
conspiracy,16230416,254,541,0.469501
canada,17330563,277,588,0.471088
Music,19110665,307,630,0.487302
europe,17760953,296,595,0.497479
gameofthrones,17883785,308,589,0.522920
hockey,19345930,337,623,0.540931


In [33]:
funny=classe_valid2.loc[train['subreddit']=='funny']
funny




,subreddit,previsao,Id,igual
35490,funny,europe,35490,0
45995,funny,worldnews,45995,0
56925,funny,funny,56925,1
15274,funny,AskReddit,15274,0
8110,funny,anime,8110,0
...,...,...,...,...
41342,funny,europe,41342,0
8017,funny,funny,8017,1
33673,funny,funny,33673,1
41983,funny,soccer,41983,0


In [34]:
errou=funny.loc[funny['igual']==0]
errou=errou.join(train,lsuffix='_caller', rsuffix='_other')
errou=errou[['Id','subreddit_caller','previsao','comment']]
errou_comment=errou['comment']
errou_comment

35490    Exactly. I see a lot of people say stuff like,...
45995    The obesity I 100% agree with, but Johnson is ...
15274    I think you're misunderstanding the purpose of...
8110     Oh God why did you show me that? I could've go...
54044    Oh no!\n\nOn a side note, I'm really impressed...
                               ...                        
22580    Dear Bank of America,  \nLast week, Darkmuscle...
51232    It appears he only became aware of the fake th...
41342                       You are now banned from r/Sino
41983    Is that... Ponce De Leon’s mythical fountain o...
54302                            This is russian I knew it
Name: comment, Length: 423, dtype: object

In [35]:
errou_comment

35490    Exactly. I see a lot of people say stuff like,...
45995    The obesity I 100% agree with, but Johnson is ...
15274    I think you're misunderstanding the purpose of...
8110     Oh God why did you show me that? I could've go...
54044    Oh no!\n\nOn a side note, I'm really impressed...
                               ...                        
22580    Dear Bank of America,  \nLast week, Darkmuscle...
51232    It appears he only became aware of the fake th...
41342                       You are now banned from r/Sino
41983    Is that... Ponce De Leon’s mythical fountain o...
54302                            This is russian I knew it
Name: comment, Length: 423, dtype: object

In [ ]:
#criacao da VAR TOTAL OF WORDS

errou['comment'].count()


errou['totalwords'] = errou['comment'].str.count(' ') + 1

errou


In [ ]:
errou['totalwords'].describe()

In [ ]:
bons=funny.loc[funny['igual']==1]
bons=bons.join(train,lsuffix='_caller', rsuffix='_other')
bons=bons[['Id','subreddit_caller','previsao','comment']]


bons['comment'].count()


bons['totalwords'] = bons['comment'].str.count(' ') + 1

bons


bons['totalwords'].describe()

In [ ]:
bons

In [ ]:
#preprocessing



#clean
bons['prep'] = bons['comment'].str.replace(r'[^\w\s]+', '') # tira td que nao palavra e espaco em branco
bons['prep'] = bons['prep'].str.lower()
bons['prep'] = bons['prep'].str.replace('(\d+)', ' num ')
bons['prep'] = bons['prep'].str.replace(r'http(?<=http).*', ' ')
bons['prep'] = bons['prep'].str.replace(r'\s+', " ")
bons['prep'] = bons['prep'].str.replace(" +", " ")






#clean
errou['prep'] = errou['comment'].str.replace(r'[^\w\s]+', '')
errou['prep'] = errou['prep'].str.lower()
errou['prep'] = errou['prep'].str.replace('(\d+)', ' num ')
errou['prep'] = errou['prep'].str.replace(r'http(?<=http).*', ' ')
errou['prep'] = errou['prep'].str.replace(r'\s+', " ")
errou['prep'] = errou['prep'].str.replace(" +", " ")

In [ ]:
lemmatizer = WordNetLemmatizer()
tt = TweetTokenizer()
def lemmatize_col(row):
    row = tt.tokenize(row)
    return ' '.join([lemmatizer.lemmatize(w) for w in row])

bons['prep'] = bons['prep'].apply(lemmatize_col)
errou['prep'] = errou['prep'].apply(lemmatize_col) 

# stopwords
stop = stopwords.words('english')
bons['prep'] = bons['prep'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
errou['prep'] = errou['prep'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
bons_words[0:10]

In [ ]:
errou_words=errou['prep'].str.split(expand=True).stack().value_counts()
errou_words[0:50]

In [ ]:
#gerar arquivo 

previsoes_test = pd.DataFrame(previsoes_test, columns=['Category'])

previsoes_test['Id'] = previsoes_test.index

previsoes_test = previsoes_test.reindex(columns=['Id', 'Category'])

previsoes_test.to_csv('linear_svc_17jul_2.csv', index=False)


